# 1 - install dependencies

In [1]:
import numpy as np
import cv2 
import os
import mediapipe as mp
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt
import time
import pandas as pd

# 2 - keypoints extractions and drawing

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

num_hand_marks = 21
num_pose_marks = 33


# holistic model process image and return the results as keypoints
def mediapipe_detection(image,model):
    image  = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image  = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results
    
def draw_styled_landmarks(image,results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    

# read the keypoints and extract them and process them
def extract_keypoints(results):
    # extract pose marks
    if results.pose_landmarks:
        pose = np.array([ [res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark ]).flatten()
    else:
        pose = np.zeros(num_pose_marks*4)
    
    # extract left hand
    if results.left_hand_landmarks:
        left_hand = np.array([ [res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark ]).flatten()
    else:
        left_hand = np.zeros(num_hand_marks*3)
        
        
    # extract right hand
    if results.right_hand_landmarks:
        right_hand = np.array([ [res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark ]).flatten()
    else:
        right_hand = np.zeros(num_hand_marks*3)
    
    return np.concatenate([pose,left_hand,right_hand])
    

# 4 - read and process data

In [3]:
train_labels = pd.read_csv("data/train_labels.csv",names=['sample','id'])
class_id = pd.read_csv("data/class_id.csv")


    
Data_path = os.path.join("data","train")



In [4]:
def check_file(file_path):
    try:
        f = open(file_path)
        f.close()
        return True
    except IOError:
        return False
    
    

def construct_path(file):
    return os.path.join(Data_path,file+"_color.mp4")
    

### 4.1 collect data that in classes 0,1,2 and available in the data/train directory

In [16]:
def get_data(id):
    data =  train_labels[train_labels['id']==id]
    lis =  [construct_path(i) for i in  (data['sample'])]
    data =  [i for i in lis if check_file(i)]
    return data,[id for i in data]
    
    

In [5]:
actions = ['sister','hurry','hungry']
n_actions = len(actions)
dic = {"sister":0,"hurry":1,"hungry":2}

In [6]:


x = [] 
y = []

for i in range(n_actions):
    data,labels = get_data(i)
    x.extend(data)
    y.extend(labels)

    
train = x 
labels = y

NameError: name 'get_data' is not defined

### 4.2 process videos into frames and label to cateogries

In [18]:


def capture_frames(video_path):
    video = cv2.VideoCapture(video_path)
    video_length = int(video.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    # count = 0
    # frame_rate = video.get(cv2.CAP_PROP_FRAME_COUNT)

    count=0
    frames = []
    while video.isOpened():
      # frame_id = video.get(0)
        ret, frame = video.read()
        
        if not ret:
            continue
        frames.append(frame)
        count += 1
        # If there are no more frames left
        if (count > (video_length-1)):
          # Release the feed
          video.release()
        # if frame_id % math.floor(1) == 0:
        #     file_name = f"{video_path}_frame_{count}.jpg"
        #     count += 1
        #     # print(f"File name: {file_name}")
        #     frames.append(frame)
        #     # cv2.imwrite(file_name, frame)
    video.release()
    return np.array(frames)


def get_frames(video_path,num_frames):
    frames = capture_frames(video_path)
    video_length = len(frames)
    steps = video_length/num_frames
    count=0
    new_frames=[]
    while count<video_length:
        frame = frames[int(count)]
#         frame = cv2.resize(frame, (256, 256))
#         frame =  (frame/255.0).astype(np.float16)
        new_frames.append(frame)
        count+=steps

    s = np.array(new_frames[:num_frames])
    print("SHAPE",s.shape)
    return np.array(new_frames[:num_frames])


In [6]:
def extract_keypoints_video(path):
    frames = get_frames(path,20)
    output_key_points=[]
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for frame in frames:
            image, results = mediapipe_detection(frame, holistic)
            output_key_points.append(extract_keypoints(results))
    return np.array(output_key_points)
    

In [157]:
train = np.array([extract_keypoints_video(i) for i in x ])





SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 512, 3)
SHAPE (20, 512, 

In [20]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [159]:
labels = to_categorical(y)

In [160]:
X_train, X_test, Y_train, Y_test = train_test_split(train, labels, test_size=0.05)

In [161]:
print(
X_train.shape,
X_test.shape,
Y_train.shape,
Y_test.shape,
)

(79, 20, 258) (5, 20, 258) (79, 3) (5, 3)


# 7 - build the model

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)



In [8]:
input_layer = Input(shape=(20,258))
layer = LSTM(64,return_sequences=True,activation="relu")(input_layer)
layer = LSTM(128,return_sequences=True,activation="relu")(layer)
layer = LSTM(64,return_sequences=False,activation="relu")(layer)
layer = Dense(64,activation="relu")(layer)
layer = Dense(32,activation="relu")(layer)
layer = Dense(len(actions),activation="softmax")(layer)

model = Model(inputs=input_layer,outputs=layer)

2021-12-09 20:48:32.743379: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-09 20:48:32.747784: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-09 20:48:32.748142: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-09 20:48:32.748859: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20, 258)]         0         
                                                                 
 lstm (LSTM)                 (None, 20, 64)            82688     
                                                                 
 lstm_1 (LSTM)               (None, 20, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99    

In [10]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [168]:
model.fit(X_train, Y_train, epochs=2000, callbacks=[tb_callback],validation_data=(X_test,Y_test))

Epoch 1/2000
3/3 [==============================] - 2s 187ms/step - loss: 1.0939 - categorical_accuracy: 0.2911 - val_loss: 1.1037 - val_categorical_accuracy: 0.2000
Epoch 2/2000
3/3 [==============================] - 0s 56ms/step - loss: 1.0558 - categorical_accuracy: 0.4557 - val_loss: 0.9858 - val_categorical_accuracy: 0.6000
Epoch 3/2000
3/3 [==============================] - 0s 55ms/step - loss: 0.8729 - categorical_accuracy: 0.5949 - val_loss: 1.5721 - val_categorical_accuracy: 0.6000
Epoch 4/2000
3/3 [==============================] - 0s 56ms/step - loss: 1.6709 - categorical_accuracy: 0.4051 - val_loss: 1.2174 - val_categorical_accuracy: 0.6000
Epoch 5/2000
3/3 [==============================] - 0s 54ms/step - loss: 0.9349 - categorical_accuracy: 0.4937 - val_loss: 0.8838 - val_categorical_accuracy: 1.0000
Epoch 6/2000
3/3 [==============================] - 0s 55ms/step - loss: 0.9753 - categorical_accuracy: 0.6329 - val_loss: 0.9411 - val_categorical_accuracy: 0.8000
Epoch 7/2

3/3 [==============================] - 0s 59ms/step - loss: 0.1367 - categorical_accuracy: 0.9494 - val_loss: 3.7857e-04 - val_categorical_accuracy: 1.0000
Epoch 100/2000
3/3 [==============================] - 0s 58ms/step - loss: 0.1831 - categorical_accuracy: 0.8987 - val_loss: 0.0314 - val_categorical_accuracy: 1.0000
Epoch 101/2000
3/3 [==============================] - 0s 60ms/step - loss: 0.2207 - categorical_accuracy: 0.8987 - val_loss: 5.8885e-04 - val_categorical_accuracy: 1.0000
Epoch 102/2000
3/3 [==============================] - 0s 58ms/step - loss: 0.0985 - categorical_accuracy: 0.9620 - val_loss: 0.0012 - val_categorical_accuracy: 1.0000
Epoch 103/2000
3/3 [==============================] - 0s 58ms/step - loss: 0.0628 - categorical_accuracy: 0.9747 - val_loss: 0.0011 - val_categorical_accuracy: 1.0000
Epoch 104/2000
3/3 [==============================] - 0s 60ms/step - loss: 0.2817 - categorical_accuracy: 0.8987 - val_loss: 0.0108 - val_categorical_accuracy: 1.0000
Epoch

Epoch 147/2000
3/3 [==============================] - 0s 55ms/step - loss: 7.3212e-06 - categorical_accuracy: 1.0000 - val_loss: 2.8610e-07 - val_categorical_accuracy: 1.0000
Epoch 148/2000
3/3 [==============================] - 0s 55ms/step - loss: 6.9470e-06 - categorical_accuracy: 1.0000 - val_loss: 2.8610e-07 - val_categorical_accuracy: 1.0000
Epoch 149/2000
3/3 [==============================] - 0s 55ms/step - loss: 6.5577e-06 - categorical_accuracy: 1.0000 - val_loss: 2.8610e-07 - val_categorical_accuracy: 1.0000
Epoch 150/2000
3/3 [==============================] - 0s 59ms/step - loss: 6.2016e-06 - categorical_accuracy: 1.0000 - val_loss: 2.6226e-07 - val_categorical_accuracy: 1.0000
Epoch 151/2000
3/3 [==============================] - 0s 59ms/step - loss: 5.9888e-06 - categorical_accuracy: 1.0000 - val_loss: 2.6226e-07 - val_categorical_accuracy: 1.0000
Epoch 152/2000
3/3 [==============================] - 0s 57ms/step - loss: 5.7052e-06 - categorical_accuracy: 1.0000 - val_lo

Epoch 194/2000
3/3 [==============================] - 0s 56ms/step - loss: 2.1623e-06 - categorical_accuracy: 1.0000 - val_loss: 9.5367e-08 - val_categorical_accuracy: 1.0000
Epoch 195/2000
3/3 [==============================] - 0s 56ms/step - loss: 2.1186e-06 - categorical_accuracy: 1.0000 - val_loss: 9.5367e-08 - val_categorical_accuracy: 1.0000
Epoch 196/2000
3/3 [==============================] - 0s 59ms/step - loss: 2.0899e-06 - categorical_accuracy: 1.0000 - val_loss: 9.5367e-08 - val_categorical_accuracy: 1.0000
Epoch 197/2000
3/3 [==============================] - 0s 55ms/step - loss: 2.0356e-06 - categorical_accuracy: 1.0000 - val_loss: 9.5367e-08 - val_categorical_accuracy: 1.0000
Epoch 198/2000
3/3 [==============================] - 0s 59ms/step - loss: 2.0009e-06 - categorical_accuracy: 1.0000 - val_loss: 9.5367e-08 - val_categorical_accuracy: 1.0000
Epoch 199/2000
3/3 [==============================] - 0s 56ms/step - loss: 1.9586e-06 - categorical_accuracy: 1.0000 - val_lo

KeyboardInterrupt: 

In [169]:
model.save_weights("model.h5")

In [11]:
model.load_weights('model.h5')

# 8 - Make predictions

# 9 - save weights

# 10 - confusion matrix evaluations

# 11 - test in real time

In [12]:
from scipy import stats

colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame


In [13]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        frame = cv2.resize(frame, (512, 512))

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-20:]
        
        if len(sequence) == 20:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-2:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'

hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOu

<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'

hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry
<class 'mediapipe.python.solution_base.SolutionOutputs'>
hurry


In [23]:
cap.release()

In [24]:
cv2.destroyAllWindows()